In [3]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\praty\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [6]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\praty\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [1]:
from transformers import pipeline
import torch
import torch.nn.functional as F
# classifier = pipeline("zero-shot-classification",
#                       model="facebook/bart-large-mnli")
# classifier(premise, hypothesis)

c:\Users\praty\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'premise' is not defined

In [21]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

device = "cpu"
text = "We should always try to eat healthy"
lables = ['travel', 'cooking', 'dancing','food']
results = {"lables": [], "score" : []}

for lable in lables:
    hypothesis = f'This example is {lable}.'
    # run through model pre-trained on MNLI
    x = tokenizer.encode(text, hypothesis, return_tensors='pt',
                        truncation_strategy='only_first')
    model =nli_model(x.to(device))
    logits = nli_model(x.to(device))[0]

    # we throw away "neutral" (dim 1) and take the probability of
    # "entailment" (2) as the probability of the label being true 
    entail_contradiction_logits = logits[:,[0,2]]
    probs = entail_contradiction_logits.softmax(dim=1)
    prob_label_is_true = probs[:,1]
    results["lables"].append(lable)
    results["score"].append(float(probs[:,1].detach().numpy()[0]))
    

classifier = pipeline("zero-shot-classification",
                      model=nli_model, tokenizer=tokenizer)
classifier(text, lables)

c:\Users\praty\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:2869: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(


{'sequence': 'We should always try to eat healthy',
 'labels': ['food', 'cooking', 'travel', 'dancing'],
 'scores': [0.924102783203125,
  0.05158378928899765,
  0.012863397598266602,
  0.011450021527707577]}

In [22]:
results["score"]

[0.00018880754942074418,
 0.17667818069458008,
 0.001159883220680058,
 0.9525015950202942]

In [28]:
tokenizer.save_pretrained(f"D:/Study_Materials/model/text_classification/model/tokenizer")
nli_model.save_pretrained(f"D:/Study_Materials/model/text_classification/model/tokenizer")

In [31]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

mod = AutoModelForSequenceClassification.from_pretrained('D:/Study_Materials/model/text_classification/model/tokenizer')
tok = AutoTokenizer.from_pretrained('D:/Study_Materials/model/text_classification/model/tokenizer')

device = "cpu"
text = "We should always try to eat healthy"
lables = ['travel', 'cooking', 'dancing','food']
results = {"lables": [], "score" : []}

for lable in lables:
    hypothesis = f'This example is {lable}.'
    # run through model pre-trained on MNLI
    x = tok.encode(text, hypothesis, return_tensors='pt',
                        truncation_strategy='only_first')
    model =mod(x.to(device))
    logits = mod(x.to(device))[0]

    # we throw away "neutral" (dim 1) and take the probability of
    # "entailment" (2) as the probability of the label being true 
    entail_contradiction_logits = logits[:,[0,2]]
    probs = entail_contradiction_logits.softmax(dim=1)
    prob_label_is_true = probs[:,1]
    results["lables"].append(lable)
    results["score"].append(float(probs[:,1].detach().numpy()[0]))
    

classifier = pipeline("zero-shot-classification",
                      model=mod, tokenizer=tok)
classifier(text, lables)

c:\Users\praty\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:2869: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(


{'sequence': 'We should always try to eat healthy',
 'labels': ['food', 'cooking', 'travel', 'dancing'],
 'scores': [0.924102783203125,
  0.05158378928899765,
  0.01286340318620205,
  0.0114500243216753]}

In [32]:
results

{'lables': ['travel', 'cooking', 'dancing', 'food'],
 'score': [0.00018880754942074418,
  0.1766781508922577,
  0.0011598836863413453,
  0.9525015950202942]}